In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.datasets
import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder

# import mnist digits dataset
mnist = tf.keras.datasets.mnist # Object of the MNIST dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()

2023-05-04 12:54:37.274867: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 12:54:37.319497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 12:54:37.320557: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 12:54:38.149727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
# # DATA TRUNCATION FOR TEST
# x_train = x_train[:100]
# y_train = y_train[:100]
# x_test = x_test[:100]
# y_test = y_test[:100]

In [3]:
# Normalize the train dataset
x_train = tf.keras.utils.normalize(x_train, axis=1)
# Normalize the test dataset
x_test = tf.keras.utils.normalize(x_test, axis=1)



# Hilbert Curve translation

In [4]:
# Hilbert path generator
# ALL CREDIT TO: https://github.com/jakubcerveny/gilbert/blob/master/gilbert2d.py
# ALL CREDIT TO @jakubcerveny on GitHub

def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)

def generate_path(x,y,ax,ay,bx,by):
    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate_path(x, y, ax2, ay2, bx, by)
        yield from generate_path(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate_path(x, y, bx2, by2, ax2, ay2)
        yield from generate_path(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate_path(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

def hilbert_path(n):
    yield from generate_path(0,0,n,0,0,n)


In [5]:
def dim_reduction(image):
    # Translate 2D image into 1D vector using Hilbert curve
    # image: 2D numpy array (28x28) or (nxn)
    # return: 1D numpy array
    width = image.shape[0]
    
    # Generate Hilbert curve path
    path = hilbert_path(width)
    # 1D vector widthxwidth
    vector = np.zeros(width*width)
    # Fill vector with image values
    for i, (x, y) in enumerate(path):
        vector[i] = image[x][y]
    return vector


In [6]:
# Reduce 28x28 images to 14x14 images by averaging pixel values in 2x2 blocks
def windowed_reduction(image):
    # image: 2D numpy array (28x28) or (nxn)
    # return: 2D numpy array (14x14)
    width = image.shape[0]
    # 2D array 14x14
    reduced = np.zeros((width//2, width//2))
    # Fill reduced with averages of 2x2 blocks
    for i in range(0, width, 2):
        for j in range(0, width, 2):
            reduced[i//2][j//2] = (image[i][j] + image[i+1][j] + image[i][j+1] + image[i+1][j+1]) / 4
    return reduced

# display first image in dataset
# plt.imshow(x_train[0], cmap=plt.cm.binary)

x_train = np.array([windowed_reduction(image) for image in x_train])
x_test = np.array([windowed_reduction(image) for image in x_test])

# plt.imshow(x_train[0], cmap=plt.cm.binary)

### Translating MNIST

Note: This takes a long time!

In [7]:
x_train = np.array([dim_reduction(x) for x in x_train])
x_test = np.array([dim_reduction(x) for x in x_test])




In [8]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(60000, 196) (10000, 196)
(60000,) (10000,)


In [14]:
print(x_train.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

# One-hot encode labels
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

(60000, 196)


# Recurrent Neural Network

Problems w/ time series classification:
- LSTM RNN Just takes windows of data and does recognition based off of that
- Taking a sufficiently long window is literally just taking a snapshot of the image and running it thru RNN instead of CNN
- wtf bro lmao.

In [17]:
model = tf.keras.models.Sequential()
model.add(
    # keras.layers.Bidirectional(
      # keras.layers.SimpleRNN(
      #     units=128,
      #     input_shape=[x_train.shape[1], 1]
      # )
      keras.layers.LSTM(
          units=128,
          input_shape=[x_train.shape[1], 1]
      )
    # )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

2023-05-03 21:15:56.679930: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 21:15:56.681470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 21:15:56.682607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/20


2023-05-03 21:16:00.997295: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 21:16:00.999019: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 21:16:01.001919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1688/1688 [==============================] - ETA: 0s - loss: 1.6517 - acc: 0.4104

2023-05-03 21:18:52.221509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 21:18:52.223109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 21:18:52.225300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1688/1688 [==============================] - 178s 105ms/step - loss: 1.6517 - acc: 0.4104 - val_loss: 1.3802 - val_acc: 0.5003
Epoch 2/20
1688/1688 [==============================] - 185s 109ms/step - loss: 1.2402 - acc: 0.5739 - val_loss: 1.0886 - val_acc: 0.6260
Epoch 3/20
1688/1688 [==============================] - 176s 104ms/step - loss: 1.0818 - acc: 0.6296 - val_loss: 0.9117 - val_acc: 0.6855
Epoch 4/20
1688/1688 [==============================] - 176s 104ms/step - loss: 0.9117 - acc: 0.6969 - val_loss: 0.7057 - val_acc: 0.7718
Epoch 5/20
1688/1688 [==============================] - 181s 107ms/step - loss: 0.7537 - acc: 0.7571 - val_loss: 0.4937 - val_acc: 0.8367
Epoch 6/20
1688/1688 [==============================] - 180s 107ms/step - loss: 0.6193 - acc: 0.8045 - val_loss: 0.4892 - val_acc: 0.8482
Epoch 7/20
1688/1688 [==============================] - 197s 117ms/step - loss: 0.5394 - acc: 0.8316 - val_loss: 0.3923 - val_acc: 0.8753
Epoch 8/20
1688/1688 [=======================

In [1]:
# test on testing set

test_loss, test_acc = model.evaluate(x_test, y_test)
9
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

history_dict = history.history

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# plot acc, val_acc
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', color='red', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()


NameError: name 'model' is not defined